In [313]:
import csv
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [314]:
%cd /content/drive/MyDrive/sjsu-aiml-project/Code

/content/drive/MyDrive/sjsu-aiml-project/Code


In [315]:
ingredientLabels = pd.read_excel('/content/drive/MyDrive/sjsu-aiml-project/Code/Joe_IngredientLabelTest.xlsx')
ingredientLabels

,Ingredient,Entity
0,a1 sauce,SAUCE
1,adobo sauce,SAUCE
2,barbecue sauce,SAUCE
3,tabasco sauce,CONDIMENT
4,tomato sauce,TOMATO
...,...,...
450,apricot,FRUIT
451,strawberry,FRUIT
452,tiramisu,DESSERT
453,hato mugi,GRAIN


In [316]:
top215 = pd.read_excel('/content/drive/MyDrive/sjsu-aiml-project/Code/Top215_df_ingredient_substitutions_w_scores_230401.xlsx')
top215

,Ingredient,Substitutes,semantic_substitutions,precision,recall,f1,accuracy
0,a1 sauce,"['barbecue sauce', 'ketchup']","['piri piri sauce', 'tomato sauce']",0.0,0.0,0.0,0.0
1,adobo sauce,['tabasco sauce'],['piri piri sauce'],0.0,0.0,0.0,0.0
2,alphonso olives,"['kalamata olives', 'gaeta olives']","['lucque olives', 'atalanta olives']",0.0,0.0,0.0,0.0
3,aluminum foil,"['plastic wrap', 'wax paper']","['plastic wrap', 'scrapple']",0.5,0.5,0.5,0.5
4,amphissa olives,"['kalamata olives', 'gaeta olives']","['atalanta olives', 'kalamata olives']",0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
209,whole-grain mustard,"['dijon mustard', 'creole mustard']","['prepared mustard', 'mustard seeds']",0.0,0.0,0.0,0.0
210,wonton wrappers,"['egg roll wrappers', 'dumpling wrappers', 'po...","['sui kow wrappers', 'dumpling wrappers', 'pla...",0.5,0.5,0.5,0.5
211,worcestershire sauce,"['fish sauce', 'soy sauce']","['worcestshire sauce', 'piri piri sauce']",0.0,0.0,0.0,0.0
212,xo sauce,['sate sauce'],['tomato sauce'],0.0,0.0,0.0,0.0


In [317]:
for index, row in ingredientLabels.iterrows():
  top215['Substitutes'] = top215['Substitutes'].str.replace(row.Ingredient,row.Entity)
  top215['semantic_substitutions'] = top215['semantic_substitutions'].str.replace(row.Ingredient, row.Entity)

top215

,Ingredient,Substitutes,semantic_substitutions,precision,recall,f1,accuracy
0,a1 sauce,"['SAUCE', 'CONDIMENT']","['SAUCE', 'TOMATO']",0.0,0.0,0.0,0.0
1,adobo sauce,['CONDIMENT'],['SAUCE'],0.0,0.0,0.0,0.0
2,alphonso olives,"['CONDIMENT', 'CONDIMENT']","['CONDIMENT', 'CONDIMENT']",0.0,0.0,0.0,0.0
3,aluminum foil,"['WRAPPER', 'WRAPPER']","['WRAPPER', 'PROCESSED MEAT']",0.5,0.5,0.5,0.5
4,amphissa olives,"['CONDIMENT', 'CONDIMENT']","['CONDIMENT', 'CONDIMENT']",0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
209,whole-grain mustard,"['CONDIMENT', 'CONDIMENT']","['CONDIMENT', 'CONDIMENT']",0.0,0.0,0.0,0.0
210,wonton wrappers,"['WRAPPER', 'WRAPPER', 'WRAPPER', 'WRAPPER']","['WRAPPER', 'WRAPPER', 'WRAPPER', 'WRAPPER']",0.5,0.5,0.5,0.5
211,worcestershire sauce,"['FERMENTED FOOD', 'FERMENTED FOOD']","['SAUCE', 'SAUCE']",0.0,0.0,0.0,0.0
212,xo sauce,['CONDIMENT'],['TOMATO'],0.0,0.0,0.0,0.0


In [318]:
def difference(string1_truth, string2_semSub):
  # initialize stuff
  true_positives = 0
  false_negatives = 0
  false_positives = 0
  metrics_list = []
  precision = 0
  recall = 0
  F1 = 0
  
  # clean strings
  string1_truth = [''.join(letter for letter in word if letter not in string.punctuation) for word in string1_truth.split(',')]
  string2_semSub = [''.join(letter for letter in word if letter not in string.punctuation) for word in string2_semSub.split(',')]
  
  # compare semSub to truth
  for semSub in string2_semSub:
    if semSub in string1_truth:
      true_positives += 1
    else:
      false_positives += 1
  
  # compare truth to semSub
  for truth in string1_truth:
    if truth not in string2_semSub:
      false_negatives += 1

  # Calculate Precision = TruePositives / (TruePositives + FalsePositives)
  precision = true_positives / (true_positives + false_positives)

  # Calculate Recall = TruePositives / (TruePositives + FalseNegatives)
  try:
    recall = true_positives / (true_positives + false_negatives)
  except:
    recall = 0

  # F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
  try:
    F1 = 2 * ( (precision * recall) / (precision + recall) )
  except:
    F1 = 0


  metrics_list.append(precision)
  metrics_list.append(recall)
  metrics_list.append(F1)

  return metrics_list

In [319]:
dataframe_temp = pd.DataFrame(columns = ['NL_precision','NL_recall','NL_F1'])

for index, row in top215.iterrows():
  x = difference(row.Substitutes, row.semantic_substitutions)
  dataframe_temp.loc[index] = x

extracted_col1 = dataframe_temp['NL_precision']
extracted_col2 = dataframe_temp['NL_recall']
extracted_col3 = dataframe_temp['NL_F1']
  
top215 = top215.join(extracted_col1)
top215 = top215.join(extracted_col2)
top215 = top215.join(extracted_col3)

top215

,Ingredient,Substitutes,semantic_substitutions,precision,recall,f1,accuracy,NL_precision,NL_recall,NL_F1
0,a1 sauce,"['SAUCE', 'CONDIMENT']","['SAUCE', 'TOMATO']",0.0,0.0,0.0,0.0,0.5,0.5,0.5
1,adobo sauce,['CONDIMENT'],['SAUCE'],0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,alphonso olives,"['CONDIMENT', 'CONDIMENT']","['CONDIMENT', 'CONDIMENT']",0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,aluminum foil,"['WRAPPER', 'WRAPPER']","['WRAPPER', 'PROCESSED MEAT']",0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,amphissa olives,"['CONDIMENT', 'CONDIMENT']","['CONDIMENT', 'CONDIMENT']",0.5,0.5,0.5,0.5,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
209,whole-grain mustard,"['CONDIMENT', 'CONDIMENT']","['CONDIMENT', 'CONDIMENT']",0.0,0.0,0.0,0.0,1.0,1.0,1.0
210,wonton wrappers,"['WRAPPER', 'WRAPPER', 'WRAPPER', 'WRAPPER']","['WRAPPER', 'WRAPPER', 'WRAPPER', 'WRAPPER']",0.5,0.5,0.5,0.5,1.0,1.0,1.0
211,worcestershire sauce,"['FERMENTED FOOD', 'FERMENTED FOOD']","['SAUCE', 'SAUCE']",0.0,0.0,0.0,0.0,0.0,0.0,0.0
212,xo sauce,['CONDIMENT'],['TOMATO'],0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [328]:
filename = 'top215_w_labels_04092023.csv'
top215.to_csv('/content/drive/MyDrive/sjsu-aiml-project/Code/' + filename)